In [2]:
import pandas as pd
from sqlalchemy import create_engine

In [3]:
data = pd.read_csv("Data/data.csv")
genre_data = pd.read_csv("Data/data_w_genres.csv")
year_data = pd.read_csv("Data/data_by_year.csv")

In [4]:
organized_data = data[["id",
                       "name",
                       "artists",
                       "year",
                       "duration_ms",
                       "explicit",
                       "acousticness",
                       "danceability",
                       "energy",
                       "instrumentalness",
                       "liveness",
                       "loudness",
                       "popularity",
                       "speechiness",
                       "tempo"                     
                      ]]

In [5]:
organized_data.to_csv("Data/CleanData.csv", index=False, header=True)

PermissionError: [Errno 13] Permission denied: 'Data/CleanData.csv'

In [7]:
engine = create_engine('sqlite:///Spotify.db', echo=True)
sqlite_connection = engine.connect()

2020-08-29 13:57:46,906 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-08-29 13:57:46,910 INFO sqlalchemy.engine.base.Engine ()
2020-08-29 13:57:46,912 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-08-29 13:57:46,914 INFO sqlalchemy.engine.base.Engine ()


In [9]:
spotify_table = "Spotify_Data"
organized_data.to_sql(spotify_table, sqlite_connection, if_exists='fail')

2020-08-29 14:00:57,202 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("Spotify_Data")
2020-08-29 14:00:57,204 INFO sqlalchemy.engine.base.Engine ()
2020-08-29 14:00:57,206 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("Spotify_Data")
2020-08-29 14:00:57,208 INFO sqlalchemy.engine.base.Engine ()
2020-08-29 14:00:57,212 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE "Spotify_Data" (
	"index" BIGINT, 
	id TEXT, 
	name TEXT, 
	artists TEXT, 
	year BIGINT, 
	duration_ms BIGINT, 
	explicit BIGINT, 
	acousticness FLOAT, 
	danceability FLOAT, 
	energy FLOAT, 
	instrumentalness FLOAT, 
	liveness FLOAT, 
	loudness FLOAT, 
	popularity BIGINT, 
	speechiness FLOAT, 
	tempo FLOAT
)


2020-08-29 14:00:57,213 INFO sqlalchemy.engine.base.Engine ()
2020-08-29 14:00:57,223 INFO sqlalchemy.engine.base.Engine COMMIT
2020-08-29 14:00:57,226 INFO sqlalchemy.engine.base.Engine CREATE INDEX "ix_Spotify_Data_index" ON "Spotify_Data" ("index")
2020-08-29 14:00:57,229 INFO sqlalchemy.eng